# data split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, json, pandas as pd, numpy as np
np.random.seed(42)
file_list = list(os.listdir('drive/MyDrive/kits_split'))
np.random.shuffle(file_list)

In [ ]:
cases = list(range(300))
np.random.shuffle(cases)
train_set = cases[:int(len(cases)*0.8)]
val_set = cases[int(len(cases)*0.8):]
print(len(train_set), len(val_set))

240 60


In [ ]:
train_set = ["case_"+str("00000"+str(i))[-5:] for i in train_set]
val_set = ["case_"+str("00000"+str(i))[-5:] for i in val_set]
train_filenames = [i for i in file_list if i[:10] in train_set]
val_filenames = [i for i in file_list if i[:10] in val_set]
print(train_filenames[:5], val_filenames[:5])

['case_00276_adj_6_seg.nii.gz', 'case_00071_adj_6_img.nii.gz', 'case_00151_random_5_img.nii.gz', 'case_00213_adj_1_seg.nii.gz', 'case_00103_adj_9_seg.nii.gz'] ['case_00293_random_4_seg.nii.gz', 'case_00063_adj_3_seg.nii.gz', 'case_00091_random_0_img.nii.gz', 'case_00086_adj_2_img.nii.gz', 'case_00091_random_3_seg.nii.gz']


In [ ]:
json.dump({"train_files":train_filenames, "val_files":val_filenames}, open("drive/MyDrive/kits_training_split.json", "w"))

# Dependencies

In [ ]:
!pip install torchio[plot]
!pip install -q monai==0.5.2
!pip install -q pytorch-lightning==1.2.10
!pip install -q gdown==3.6.4 matplotlib==3.2.2 pandas==1.1.5 seaborn==0.11.1
!git clone https://github.com/srg9000/surface-distance.git
!pip install surface-distance/ --force-reinstall

# Fetch Data

In [3]:
!cp -r ./drive/MyDrive/kits_split/ .
!cp ./drive/MyDrive/kits_training_split.json .

# Imports

In [14]:
import os, sys
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import nibabel as nib
import torch
import torch.nn as nn
from torch.utils.data import random_split, DataLoader, Dataset
import monai
import json
from datetime import datetime
import pandas as pd
import torchio as tio
# import surface_distance
import pytorch_lightning as pl
import gc
# from surface_distance import compute_surface_distances
import seaborn as sns; sns.set()
sns.set_style("whitegrid", {'axes.grid' : False})
%config InlineBackend.figure_format = 'retina'
torch.manual_seed(14041931)
print('TorchIO version:', tio.__version__)

TorchIO version: 0.18.47


# Data loader and preprocessing

In [17]:
file_data = json.load(open('kits_training_split.json'))
train_filenames = file_data['train_files']
val_filenames = file_data['val_files']

In [45]:
class KitsDataset(Dataset, pl.LightningDataModule):
    def __init__(self, root_dir, filenames, mode):
        self.root_dir = root_dir
        self.filenames = filenames
        self.data = self.filenames # list(os.listdir(self.root_dir))
        self.data_input = [i for i in self.data if 'img' in i]
        self.data_labels = [i.replace('img', 'seg') for i in self.data_input]
        self.mode = mode
        self.preprocess = self.get_preprocessing_transform()
        self.augment = self.get_augmentation_transform()
        if mode == 'train':
            self.transform = tio.Compose([self.preprocess, self.augment])
        else:
            self.transform = self.preprocess
    
    def __len__(self):
        return len(self.filenames)

    def get_preprocessing_transform(self):
        preprocess = tio.Compose([
            tio.RescaleIntensity((-1, 1)),
            # tio.CropOrPad((512, 512)),
            tio.EnsureShapeMultiple(8),  # for the U-Net
            tio.OneHot(num_classes=4),
            
        ])
        return preprocess
        
    def get_augmentation_transform(self):
        augment = tio.Compose([
            tio.RandomAffine(p=0.2),
            tio.RandomGamma(p=0.4),
            tio.RandomNoise(p=0.4),
            tio.OneOf([
                tio.RandomMotion(p=0.3),
                tio.RandomBiasField(p=0.25),
                tio.RandomGhosting(p=0.2)
            ]),
            tio.OneOf([
                tio.RandomMotion(p=0.2),
                tio.RandomSpike(p=0.2)
            ])
        ])
        return augment

    def __getitem__(self, idxs):
        subjects = []
        if isinstance(idxs, int):
            idxs = [idxs]
        for idx in idxs:
            if self.mode == 'test':
                subject = tio.Subject(
                    image=tio.ScalarImage(self.root_dir+self.data_input[idx])
                )
            else:
                subject = tio.Subject(
                    image=tio.ScalarImage(self.root_dir+self.data_input[idx]),
                    label=tio.LabelMap(self.root_dir+self.data_labels[idx])
                )
            subjects.append(subject)
        return subjects[0]

In [46]:
# Create custom dataloader, load all files dynamically
class KitsDataModule(pl.LightningDataModule):
    def __init__(self, root_dir='./kits_split/', batch_size=4, train_filenames=[], val_filenames=[]):
        super().__init__()
        self.root_dir = root_dir
        self.batch_size = batch_size
        self.train_filenames = train_filenames
        self.val_filenames = val_filenames
        self.test_filenames = val_filenames
        self.train_set = KitsDataset(self.root_dir, self.train_filenames, mode='train')
        self.val_set = KitsDataset(self.root_dir, self.val_filenames, mode='val')
        self.test_set = KitsDataset(self.root_dir, self.val_filenames, mode='test')

    # def get_preprocessing_transform(self):
    #     preprocess = tio.Compose([
    #         tio.RescaleIntensity((-1, 1)),
    #         tio.CropOrPad(self.get_max_shape(self.subjects + self.test_subjects)),
    #         tio.EnsureShapeMultiple(8),  # for the U-Net
    #         tio.OneHot(),
    #     ])
    #     return preprocess
    
    # def get_augmentation_transform(self):
    #     augment = tio.Compose([
    #         tio.RandomAffine(),
    #         # tio.RandomGamma(p=0.5),
    #         # tio.RandomNoise(p=0.5),
    #         # tio.RandomMotion(p=0.1),
    #         # tio.RandomBiasField(p=0.25),
    #     ])
    #     return augment

    def train_dataloader(self):
        return DataLoader(self.train_set, self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, self.batch_size)
    

In [47]:
data = KitsDataModule(
    root_dir='./kits_split/',
    train_filenames = train_filenames,
    val_filenames = val_filenames,    
    batch_size=4
)

# Model definition

In [48]:
# https://github.com/luuuyi/CBAM.PyTorch/blob/master/model/resnet_cbam.py
class ChannelAttention(nn.Module):
    def __init__(self, submodule, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.submodule = submodule
        self.avg_pool = nn.AdaptiveAvgPool3d(1)
        self.max_pool = nn.AdaptiveMaxPool3d(1)
           
        self.fc = nn.Sequential(nn.Conv3d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv3d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.submodule(x)
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        print("CA output = ", out.shape, flush=True)
        return y*self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, submodule, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.submodule = submodule
        self.conv1 = nn.Conv3d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.conv2 = nn.Conv3d(4, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.gelu = nn.GELU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.submodule(x)
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        # x = self.gelu(x)
        # x = self.conv2(x)
        return y*self.sigmoid(x)

class ConvBlock(nn.Module):
    def __init__(self, in_c=32, out_c=32, kernel_size=3, activation=nn.GELU):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv3d(n_c, out_c, kernel_size, padding=kernel_size//2, bias=False)
        self.batchnorm = nn.BatchNorm3d(out_c)
        self.activation = activation()
        self.conv2 = nn.Conv3d(out_c, out_c, kernel_size, padding=kernel_size//2, bias=False)
        self.batchnorm2 = nn.BatchNorm3d(out_c)
        self.activation2 = activation()

    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.activation2(x)
        return x

In [49]:
# Copyright 2020 - 2021 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import warnings
from typing import Sequence, Tuple, Union

import torch
import torch.nn as nn

from monai.networks.blocks.convolutions import Convolution, ResidualUnit
from monai.networks.layers.factories import Act, Norm
from monai.networks.layers.simplelayers import SkipConnection
from monai.utils import alias, export

__all__ = ["UNet", "Unet", "unet"]


@export("monai.networks.nets")
@alias("Unet")
class UNet(nn.Module):
    def __init__(
        self,
        dimensions: int,
        in_channels: int,
        out_channels: int,
        channels: Sequence[int],
        strides: Sequence[int],
        kernel_size: Union[Sequence[int], int] = 3,
        up_kernel_size: Union[Sequence[int], int] = 3,
        num_res_units: int = 0,
        act: Union[Tuple, str] = Act.PRELU,
        norm: Union[Tuple, str] = Norm.INSTANCE,
        dropout=0.0,
    ) -> None:
        """
        Enhanced version of UNet which has residual units implemented with the ResidualUnit class.
        The residual part uses a convolution to change the input dimensions to match the output dimensions
        if this is necessary but will use nn.Identity if not.
        Refer to: https://link.springer.com/chapter/10.1007/978-3-030-12029-0_40.

        Args:
            dimensions: number of spatial dimensions.
            in_channels: number of input channels.
            out_channels: number of output channels.
            channels: sequence of channels. Top block first. The length of `channels` should be no less than 2.
            strides: sequence of convolution strides. The length of `stride` should equal to `len(channels) - 1`.
            kernel_size: convolution kernel size, the value(s) should be odd. If sequence,
                its length should equal to dimensions. Defaults to 3.
            up_kernel_size: upsampling convolution kernel size, the value(s) should be odd. If sequence,
                its length should equal to dimensions. Defaults to 3.
            num_res_units: number of residual units. Defaults to 0.
            act: activation type and arguments. Defaults to PReLU.
            norm: feature normalization type and arguments. Defaults to instance norm.
            dropout: dropout ratio. Defaults to no dropout.

        Note: The acceptable spatial size of input data depends on the parameters of the network,
            to set appropriate spatial size, please check the tutorial for more details:
            https://github.com/Project-MONAI/tutorials/blob/master/modules/UNet_input_size_constrains.ipynb.
            Typically, when using a stride of 2 in down / up sampling, the output dimensions are either half of the
            input when downsampling, or twice when upsampling. In this case with N numbers of layers in the network,
            the inputs must have spatial dimensions that are all multiples of 2^N.
            Usually, applying `resize`, `pad` or `crop` transforms can help adjust the spatial size of input data.

        """
        super().__init__()

        if len(channels) < 2:
            raise ValueError(
                "the length of `channels` should be no less than 2.")
        delta = len(strides) - (len(channels) - 1)
        if delta < 0:
            raise ValueError(
                "the length of `strides` should equal to `len(channels) - 1`.")
        if delta > 0:
            warnings.warn(f"`len(strides) > len(channels) - 1`, the last {delta} values of strides will not be used.")
        if isinstance(kernel_size, Sequence):
            if len(kernel_size) != dimensions:
                raise ValueError(
                    "the length of `kernel_size` should equal to `dimensions`.")
        if isinstance(up_kernel_size, Sequence):
            if len(up_kernel_size) != dimensions:
                raise ValueError(
                    "the length of `up_kernel_size` should equal to `dimensions`.")

        self.dimensions = dimensions
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.channels = channels
        self.strides = strides
        self.kernel_size = kernel_size
        self.up_kernel_size = up_kernel_size
        self.num_res_units = num_res_units
        self.act = act
        self.norm = norm
        self.dropout = dropout
        # self.ca1 = ChannelAttention(16)
        # self.ca1 = ChannelAttention(16)
        # self.sa1 = SpatialAttention()
        # self.sa2 = SpatialAttention()
        # self.sa3 = SpatialAttention()

        def _create_block(
            inc: int,
            outc: int,
            channels: Sequence[int],
            strides: Sequence[int],
            is_top: bool) -> nn.Sequential:
            """
            Builds the UNet structure from the bottom up by recursing down to the bottom block, then creating sequential
            blocks containing the downsample path, a skip connection around the previous block, and the upsample path.

            Args:
                inc: number of input channels.
                outc: number of output channels.
                channels: sequence of channels. Top block first.
                strides: convolution stride.
                is_top: True if this is the top block.
            """
            c = channels[0]
            s = strides[0]

            subblock: nn.Module
            if len(channels) > 2:
                # continue recursion down
                subblock = _create_block(
                    c, c, channels[1:], strides[1:], False)
                subblock = SpatialAttention(subblock)
                upc = c * 2
            else:
                # the next layer is the bottom so stop recursion, create the bottom layer as the sublock for this layer
                subblock = self._get_bottom_layer(c, channels[1])
                subblock = ChannelAttention(subblock, in_planes=c)
                upc = c + channels[1]

            # create layer in downsampling path
            down = self._get_down_layer(inc, c, s, is_top)
            # create layer in upsampling path
            up = self._get_up_layer(upc, outc, s, is_top)
            return nn.Sequential(down, SkipConnection(subblock), up)

        self.model = _create_block(
            in_channels, out_channels, self.channels, self.strides, True)

    def _get_down_layer(self,
        in_channels: int,
        out_channels: int,
        strides: int,
        is_top: bool) -> nn.Module:
        """ 
        Args:
            in_channels: number of input channels.
            out_channels: number of output channels.
            strides: convolution stride.
            is_top: True if this is the top block.
        """
        if self.num_res_units > 0:
            return ResidualUnit(
                self.dimensions,
                in_channels,
                out_channels,
                strides=strides,
                kernel_size=self.kernel_size,
                subunits=self.num_res_units,
                act=self.act,
                norm=self.norm,
                dropout=self.dropout,
            )
        return Convolution(
            self.dimensions,
            in_channels,
            out_channels,
            strides=strides,
            kernel_size=self.kernel_size,
            act=self.act,
            norm=self.norm,
            dropout=self.dropout,
        )

    def _get_bottom_layer(self, in_channels: int, out_channels: int) -> nn.Module:
        """
        Args:
            in_channels: number of input channels.
            out_channels: number of output channels.
        """
        return self._get_down_layer(in_channels, out_channels, 1, False)

    def _get_up_layer(self, in_channels: int, out_channels: int, strides: int, is_top: bool) -> nn.Module:
        """
        Args:
            in_channels: number of input channels.
            out_channels: number of output channels.
            strides: convolution stride.
            is_top: True if this is the top block.
        """
        conv: Union[Convolution, nn.Sequential]

        conv = Convolution(
            self.dimensions,
            in_channels,
            out_channels,
            strides=strides,
            kernel_size=self.up_kernel_size,
            act=self.act,
            norm=self.norm,
            dropout=self.dropout,
            conv_only=is_top and self.num_res_units == 0,
            is_transposed=True,
        )

        if self.num_res_units > 0:
            ru = ResidualUnit(
                self.dimensions,
                out_channels,
                out_channels,
                strides=1,
                kernel_size=self.kernel_size,
                subunits=1,
                act=self.act,
                norm=self.norm,
                dropout=self.dropout,
                last_conv_only=is_top,
            )
            conv = nn.Sequential(conv, ru)

        return conv

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.model(x)
        return x


Unet = unet = UNet


In [59]:
class Model(pl.LightningModule):
    def __init__(self, net, criterion, learning_rate, optimizer_class):
        super().__init__()
        self.lr = learning_rate
        self.net = net
        self.criterion = criterion
        self.optimizer_class = optimizer_class
    
    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        return optimizer
    
    def prepare_batch(self, batch):
        print(batch)
        return batch['image']['data'], batch['label']['data']
    
    def infer_batch(self, batch):
        x, y = self.prepare_batch(batch)
        y_hat = self.net(x)
        return y_hat, y

    def training_step(self, batch, batch_idx):
        print(batch)
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        print(batch)
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss)
        return loss

In [60]:
unet = UNet(
    dimensions=3,
    in_channels=1,
    out_channels=3,
    channels=(8, 16, 32, 16),
    strides=(2, 2, 2), 
)

# TODO: Create custom criterion / use multiple
model = Model(
    net=unet,
    criterion=monai.losses.DiceCELoss(softmax=True),
    learning_rate=1e-3,
    optimizer_class=torch.optim.AdamW,
)

early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss',
)

trainer = pl.Trainer(
    # gpus=1,
    # precision=16,
    callbacks=[early_stopping],
    # amp_backend = ,
)

trainer.logger._default_hp_metric = False

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


# Train model

In [61]:
start = datetime.now()
print('Training started at', start)
trainer.fit(model=model, datamodule=data)
print('Training duration:', datetime.now() - start)


  | Name      | Type       | Params
-----------------------------------------
0 | net       | UNet       | 64.6 K
1 | criterion | DiceCELoss | 0     
-----------------------------------------
64.6 K    Trainable params
0         Non-trainable params
64.6 K    Total params
0.258     Total estimated model params size (MB)


Training started at 2021-08-18 06:28:29.225802


Validation sanity check: 0it [00:00, ?it/s]

{'image': {'path': ['kits_split/case_00091_random_0_img.nii.gz', 'kits_split/case_00086_adj_2_img.nii.gz', 'kits_split/case_00091_adj_8_img.nii.gz', 'kits_split/case_00091_adj_6_img.nii.gz'], 'stem': ['case_00091_random_0_img', 'case_00086_adj_2_img', 'case_00091_adj_8_img', 'case_00091_adj_6_img'], 'type': ['intensity', 'intensity', 'intensity', 'intensity']}, 'label': {'path': ['kits_split/case_00091_random_0_seg.nii.gz', 'kits_split/case_00086_adj_2_seg.nii.gz', 'kits_split/case_00091_adj_8_seg.nii.gz', 'kits_split/case_00091_adj_6_seg.nii.gz'], 'stem': ['case_00091_random_0_seg', 'case_00086_adj_2_seg', 'case_00091_adj_8_seg', 'case_00091_adj_6_seg'], 'type': ['label', 'label', 'label', 'label']}}
{'image': {'path': ['kits_split/case_00091_random_0_img.nii.gz', 'kits_split/case_00086_adj_2_img.nii.gz', 'kits_split/case_00091_adj_8_img.nii.gz', 'kits_split/case_00091_adj_6_img.nii.gz'], 'stem': ['case_00091_random_0_img', 'case_00086_adj_2_img', 'case_00091_adj_8_img', 'case_00091_a

KeyError: ignored